In [1]:
#### t_loss_bkwd2 vs t_loss_bkwd3 on train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
#from model_triton import *
from loss_and_optimizer_triton import t_loss, t_loss_bkwd3, t_loss_bkwd3_t
BS, H, N, D = 2, 2, 512, 64 # 1, 1, 3, 4 #2, 2, 5, 4
vocab_size = 128
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda").to(torch.int32)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
train=False

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
# print(mask)

res2 = t_loss_bkwd3(layers_params, y, mask, None, train, p_gen_aux)
#print(res2[1])
print_res_shapes(res2[0]) 

print(f'----XXX----')

res3 = t_loss_bkwd3_t(layers_params, y, mask, None, train, p_gen_aux)
#print(res3)
print_res_shapes(res3[0]) 


5
- 2
-- torch.Size([128, 64]) tensor([-0.003505,  0.007882, -0.001664], device='cuda:0')
-- torch.Size([128]) tensor([ 0.000964,  0.001854, -0.001920], device='cuda:0')
- 1
-- torch.Size([512, 64]) tensor([ 3.864176e-05, -6.586349e-05, -3.140170e-04], device='cuda:0')
- 10
-- torch.Size([64]) tensor([1.789831e-06, 1.210890e-05, 6.373659e-06], device='cuda:0')
-- torch.Size([64]) tensor([-3.012282e-05, -8.887264e-05,  8.828213e-05], device='cuda:0')
-- torch.Size([2, 3, 32, 64]) tensor([-2.479468e-05, -2.766168e-05,  4.145396e-05], device='cuda:0')
-- torch.Size([64, 64]) tensor([-7.565518e-05,  1.041194e-04,  5.504349e-05], device='cuda:0')
-- torch.Size([64]) tensor([-1.286771e-05,  7.743671e-05,  1.872615e-05], device='cuda:0')
-- torch.Size([64]) tensor([-1.702363e-05, -4.738345e-05, -7.780627e-05], device='cuda:0')
-- torch.Size([256, 64]) tensor([-7.358280e-06, -1.423518e-04,  1.861453e-04], device='cuda:0')
-- torch.Size([256]) tensor([-9.753904e-05,  2.259074e-04, -4.182720e-05

loc(callsite("/efs/notebooks/mkukla/pre-tjax/model_triton.py":1449:7 at "/efs/notebooks/mkukla/pre-tjax/model_triton.py":907:74)): error: operation scheduled before its operands


5
- 2
-- torch.Size([128, 64]) tensor([-0.003682,  0.005054,  0.000734], device='cuda:0')
-- torch.Size([128]) tensor([ 0.000964,  0.001854, -0.001920], device='cuda:0')
- 1
-- torch.Size([512, 64]) tensor([ 5.972455e-06, -1.167266e-05, -4.032930e-05], device='cuda:0')
- 10
-- torch.Size([64]) tensor([4.132200e-07, 1.563793e-06, 9.823967e-07], device='cuda:0')
-- torch.Size([64]) tensor([-4.424945e-06, -1.176063e-05,  1.363217e-05], device='cuda:0')
-- torch.Size([2, 3, 32, 64]) tensor([-4.407902e-06, -4.250490e-06,  6.683092e-06], device='cuda:0')
-- torch.Size([64, 64]) tensor([-1.154554e-05,  1.674224e-05,  8.602457e-06], device='cuda:0')
-- torch.Size([64]) tensor([-1.700682e-06,  1.079263e-05,  9.962064e-07], device='cuda:0')
-- torch.Size([64]) tensor([-2.067035e-06, -7.818023e-06, -1.069047e-05], device='cuda:0')
-- torch.Size([256, 64]) tensor([-1.694028e-06, -2.026383e-05,  2.860608e-05], device='cuda:0')
-- torch.Size([256]) tensor([-1.273283e-05,  3.240734e-05, -3.735968e-06